<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Sentence-embedding" data-toc-modified-id="Sentence-embedding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sentence embedding</a></span><ul class="toc-item"><li><span><a href="#Infersent" data-toc-modified-id="Infersent-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Infersent</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dimensionality reduction</a></span><ul class="toc-item"><li><span><a href="#PCA" data-toc-modified-id="PCA-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>PCA</a></span></li></ul></li><li><span><a href="#Blocking" data-toc-modified-id="Blocking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Blocking</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#bilstm,-rnn_dim-2048" data-toc-modified-id="bilstm,-rnn_dim-2048-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>bilstm, rnn_dim 2048</a></span></li><li><span><a href="#Searching" data-toc-modified-id="Searching-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Searching</a></span></li></ul></li><li><span><a href="#Best" data-toc-modified-id="Best-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Best</a></span></li></ul></div>

# Setup 

In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
params = {}

# Import dataset

In [2]:
from preprocessing_datasets.preprocessing_amzn_gp import clean_amzn_gp
table, pairs = clean_amzn_gp()

In [3]:
table

,title,description,manufacturer,price
0,clickart 950 000 - premier image pack (dvd-rom),unk,broderbund,0
1,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0
2,noah's ark activity center (jewel case ages 3-8),unk,victory multimedia,0
3,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99
...,...,...,...,...
4584,jumpstart(r) advanced 1st grade,prepare your child for the 1st grade and beyon...,unk,19.99
4585,ibm(r) viavoice(r) advanced edition 10,ibm viavoice advanced edition release 10 is a ...,unk,78.95
4586,xbox 360: gears of war,as marcus fenix you fight a war against the im...,unk,59.99
4587,documents to go premium 7.0,this pda software enables you to use your docu...,unk,49.99


# Sentence embedding

## Infersent

Setup model

In [4]:
from embedding_algorithms.inferSent import set_RNN_embedding
start = time.time()
model_type = "bilstm" 
char_level = False
model_version = 2
rnn_dim = 2048
verbose = 1
set_RNN_embedding(model_type, char_level, model_version, rnn_dim, verbose)
print("TIME: {0}".format(time.time() - start))

params['embedding'] = {
    'name': 'Infersent',
    'model_type': model_type,
    'char_level': char_level,
    'model_version': model_version,
    'rnn_dim': rnn_dim,
    'verbose': verbose
}

Vocab size : 2196017
TIME: 180.73784613609314


Embedding

In [5]:
from embedding_algorithms.inferSent import RNN_embedding
start = time.time()
attr_list = ['title', 'manufacturer']
params['attr_list'] = attr_list
embeddings_tokens = RNN_embedding(table, attr_list, model_type, char_level)
embeddings_tokens = np.array(embeddings_tokens)
print("TIME: {0}".format(time.time() - start))

attrs: ['title', 'manufacturer']
TIME: 67.96230816841125


# Dimensionality reduction

## PCA

In [6]:
from dimensionality_reduction_algorithms.pca import pca_dim_reduction

start = time.time()
pca_embeddings = pca_dim_reduction(
            embeddings_tokens, 
            num_components=2,
            verbose=1)

print("PCA: {0}".format(time.time() - start))

params['reduction'] = {
    'name': 'PCA',
    'num_components': 2,
    'verbose': 1,
}

starting dimension: 4096
setting PCA with n_components: 2
PCA: 0.5006558895111084


# Blocking

In [10]:
from cluster_algorithms.hierarchy_cluster import hierarchy_cluster
num_clusters = 3
start = time.time()
blocks = hierarchy_cluster(pca_embeddings, {'num_clusters': num_clusters})
print("BLOCKS: {0}".format(time.time() - start))

params['blocking'] = {
    'name': 'hierarchy',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 3, 
BLOCKS: 0.5847048759460449


# Evaluation

## bilstm, rnn_dim 2048

In [11]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.6335124761972976
(PC) Pair completeness is: 0.546923076923077
(RM) Reference metric (Harmonic mean RR and PC) is: 0.587041947076379
(PQ) Pair quality - Precision is: 0.00018428879687408875
(FM) Fmeasure is: 0.0003684534412955466

['title', 'manufacturer']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'PCA', 'num_components': 2, 'verbose': 1}
{'name': 'hierarchy', 'num_clusters': 3}


## Searching

In [9]:
for i in range(1,20,1):
    num_clusters = int(i)
    blocks = hierarchy_cluster(pca_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

clustering with NUM_CLUSTERS = 1, 
(RR) Reduction ratio is: 0.0
(PC) Pair completeness is: 1.0
(RM) Reference metric (Harmonic mean RR and PC) is: 0
(PQ) Pair quality - Precision is: 0.00012349002571062335
(FM) Fmeasure is: 0.0002469495556142747
clustering with NUM_CLUSTERS = 2, 
(RR) Reduction ratio is: 0.48039994809619224
(PC) Pair completeness is: 0.6269230769230769
(RM) Reference metric (Harmonic mean RR and PC) is: 0.543967400314665
(PQ) Pair quality - Precision is: 0.00014899680360722176
(FM) Fmeasure is: 0.00029792280180493695
clustering with NUM_CLUSTERS = 3, 
(RR) Reduction ratio is: 0.6335124761972976
(PC) Pair completeness is: 0.546923076923077
(RM) Reference metric (Harmonic mean RR and PC) is: 0.587041947076379
(PQ) Pair quality - Precision is: 0.00018428879687408875
(FM) Fmeasure is: 0.0003684534412955466
clustering with NUM_CLUSTERS = 4, 
(RR) Reduction ratio is: 0.686586589401174
(PC) Pair completeness is: 0.5138461538461538
(RM) Reference metric (Harmonic mean RR and P

# Best

In [28]:
# Amazon-GP
# RR PC alpha

list_hierarchy_tsne = [0.9277, 0.8407, 0.8821]
list_hierarchy_pca = [0.633, 0.546, 0.5870] # PCA: 0.50, BLOCKS: 0.584, n_cluster: 3
